In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
df = pd.read_csv("../Data/raw-data/credit_risk_dataset.csv")
df.head()

In [ ]:
df.info()

In [ ]:
df.duplicated().sum()

In [ ]:
# Find all duplicate rows
duplicates = df[df.duplicated()]

# Count of duplicates
print(f"Number of duplicate rows: {duplicates.shape[0]}")

# View duplicates
print(duplicates)


In [ ]:
duplicates.head(40)

In [ ]:
# the Percentage of duplicates are too samll it is less than 0.5% of our dataset.
# Although there is no column for customer_id to be sure wheather they are duplicates or not, the will be dropped.
df = df.drop_duplicates()
df.info()

In [ ]:
df.columns

In [ ]:
## Make the columns labels clearer:
df.rename(columns={
    'person_emp_length': 'person_employment_length',
    'loan_intent': 'loan_purpose',
    'loan_amnt': 'loan_amount',
    'loan_int_rate': 'loan_interest_rate',
    'loan_percent_income': 'loan_to_income_ratio',
    'cb_person_cred_hist_length': 'cb_credit_history_length'
}, inplace=True)


In [ ]:
df.info()

In [ ]:
# Function of Counting missing values, defining where they exist and in which column, its type, their percentage 
def missing_summary(df):
    summary = df.isnull().sum().to_frame(name='missing_count')
    summary['missing_pct'] = (summary['missing_count'] / len(df)) * 100
    summary['missing_pct'] = summary['missing_pct'].apply(lambda x: f"{x:.2f}%")
    summary['dtype'] = df.dtypes
    summary['type'] = summary['dtype'].apply(lambda x: 'numerical' if pd.api.types.is_numeric_dtype(x) else 'categorical')
    summary = summary[summary['missing_count'] > 0]
    return summary.sort_values(by='missing_count', ascending=False)
missing_summary(df)

In [ ]:
## The percentage of missing values in column loan_interest_rat is high and it is a sensitve column. 
## That is why we will anylze it deeper in order to handel them:
## Step 1: Analyze Missingness Pattern
# 1.1 Determine Missingness Mechanism
# Check if missingness is random or systematic
missing_mask = df['loan_interest_rate'].isnull()

# Compare distributions of other variables between missing and non-missing groups
for col in ['loan_grade', 'loan_status', 'person_income', 'loan_amount']:
    print(f"\nComparison for {col}:")
    print(pd.concat([
        df.loc[~missing_mask, col].describe().rename('Not Missing'),
        df.loc[missing_mask, col].describe().rename('Missing')
    ], axis=1))

In [ ]:
df.loan_grade.unique()

In [ ]:
## 1.2 Visualize Relationships
# Create a missing indicator plot
plt.figure(figsize=(10,6))
sns.heatmap(df.isnull(), cbar=False, cmap='viridis')
plt.title('Missing Values Pattern')
plt.show()

# Check relationship with loan grade
plt.figure(figsize=(10,6))
sns.boxplot(data=df, x='loan_grade', y='loan_interest_rate', hue=missing_mask)
plt.title('Interest Rate Distribution by Grade (Missing vs Present)')
plt.show()

In [ ]:
# Calculate missing rate by grade
grade_missing_rates = df.groupby('loan_grade')['loan_interest_rate'].apply(
    lambda x: x.isnull().mean()
).sort_values(ascending=False)

print("Percentage of missing interest rates by grade:")
print(grade_missing_rates)

In [ ]:
df.info()

In [ ]:
df.cb_person_default_on_file.unique()

In [ ]:
# Convert selected columns to lowercase
columns_to_fix = ['person_home_ownership', 'loan_purpose']

for col in columns_to_fix:
    df[col] = df[col].str.lower()

# Print result
print(df[columns_to_fix].head())

In [ ]:
df.loan_purpose.unique()

In [ ]:
# Mapping of incorrect to correct values
purpose_fixes = {
    'homeimprovement': 'home-improvement',
    'debtconsolidation': 'debt-consolidation'
}
# Clean the 'loan_purpose' column
df['loan_purpose'] = df['loan_purpose'].str.lower().replace(purpose_fixes)

# (Optional) Print unique values to verify
print(df['loan_purpose'].unique())

In [ ]:
df.person_home_ownership.unique()

In [ ]:
df.person_employment_length.describe()

In [ ]:
## Replace the mising values in column: person_employment_length with the mode since it has outlires
# Calculate the median (excluding NaN)
median_value = df['person_employment_length'].median()

# Fill missing values with median
df['person_employment_length'].fillna(median_value, inplace=True)

# (Optional) Check result
print(df['person_employment_length'].isnull().sum())  # Should be 0


In [ ]:
df.info()

In [ ]:
## Check the outliers in the numercial columns:
# Select numeric columns only
numeric_cols = df.select_dtypes(include='number').columns

# Loop through numeric columns and detect outliers
for col in numeric_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
    print(f"{col}: {len(outliers)} outliers")

In [ ]:
df.loan_status.unique()

In [ ]:
# Detecting outliers by boxplott
# Filter only numerical columns with more than 2 unique values (to exclude binary like loan_status)
numeric_cols = [col for col in df.select_dtypes(include='number').columns
                if df[col].nunique() > 2]

# Set up plot size
plt.figure(figsize=(15, len(numeric_cols) * 4))

# Plot each column as a separate subplot
for i, col in enumerate(numeric_cols, 1):
    plt.subplot(len(numeric_cols), 1, i)
    sns.boxplot(x=df[col])
    plt.title(f'Boxplot of {col}')
    plt.xlabel(col)

plt.tight_layout()
plt.show()


In [ ]:
# Handeling the outliers:
# Replace extreme values with percentile thresholds (e.g., 1st and 99th):

def cap_outliers(df, col, lower_q=0.01, upper_q=0.99):
    lower = df[col].quantile(lower_q)
    upper = df[col].quantile(upper_q)
    df[col] = df[col].clip(lower, upper)

for col in ['person_age', 'person_income', 'person_employment_length',
            'loan_amount', 'loan_to_income_ratio', 'cb_credit_history_length']:
    cap_outliers(df, col)

In [ ]:
# Plotting outliers after capping them. Binary values column will not be included
numeric_cols = [col for col in df.select_dtypes(include='number').columns
                if df[col].nunique() > 2]

# Set up plot size
plt.figure(figsize=(15, len(numeric_cols) * 4))

# Plot each column as a separate subplot
for i, col in enumerate(numeric_cols, 1):
    plt.subplot(len(numeric_cols), 1, i)
    sns.boxplot(x=df[col])
    plt.title(f'Boxplot of {col}')
    plt.xlabel(col)

plt.tight_layout()
plt.show()

In [ ]:
# There are two columns still suffer of a lote of outliers after capping: person_income and loan_amount. 
# Log trasnformation will be implemented on them
df['person_income_log'] = np.log1p(df['person_income'])

In [ ]:
plt.figure(figsize=(8, 5))
sns.histplot(df['person_income_log'], kde=True, bins=30)
plt.title('Histogram of Log-Transformed Person Income')
plt.xlabel('Log(Person Income)')
plt.ylabel('Count')
plt.show()

In [ ]:
plt.figure(figsize=(8, 2))
sns.boxplot(x=df['person_income_log'])
plt.title('Boxplot of Log-Transformed Person Income')
plt.xlabel('Log(Person Income)')
plt.show()

In [ ]:
df['loan_amount_log'] = np.log1p(df['loan_amount'])
plt.figure(figsize=(8, 5))
sns.histplot(df['loan_amount_log'], kde=True, bins=30)
plt.title('Histogram of Log-Transformed Loan Amount')
plt.xlabel('Log(Loan Amount)')
plt.ylabel('Count')
plt.show()

In [ ]:
plt.figure(figsize=(8, 2))
sns.boxplot(x=df['loan_amount_log'])
plt.title('Boxplot of Log-Transformed Loan Amount')
plt.xlabel('Log(Loan Amount)')
plt.show()

In [ ]:
plt.figure(figsize=(8, 5))
sns.histplot(df['loan_amount'], kde=True, bins=30)
plt.title('Histogram of Log-Transformed Loan Amount')
plt.xlabel('Log(Loan Amount)')
plt.ylabel('Count')
plt.show()

In [ ]:
plt.figure(figsize=(12, 5))
# Before
plt.subplot(1, 2, 1)
sns.histplot(df['loan_amount'], kde=True, bins=30)
plt.title('Original Loan Amount')

# After
plt.subplot(1, 2, 2)
sns.histplot(df['loan_amount_log'], kde=True, bins=30)
plt.title('Log-Transformed Loan Amount')

plt.tight_layout()
plt.show()


In [ ]:
df.info()

In [ ]:
df.loan_grade.value_counts()

In [ ]:
## Regard the categorical columns. All of them looks fine except of loan_grade where there are very small categories.
## The small categories E, F, G will be groubed together in one new category: E_or_lower:
df['loan_grade'] = df['loan_grade'].replace({'E': 'E_or_lower', 'F': 'E_or_lower', 'G': 'E_or_lower'})
df.loan_grade.value_counts()

In [ ]:
df.info()

In [ ]:
# Data type valedation and change
# Cast object columns to category
categorical_cols = [
    'person_home_ownership', 
    'loan_purpose', 
    'loan_grade', 
    'cb_person_default_on_file'
]
df[categorical_cols] = df[categorical_cols].astype('category')

# convert binary column to bool or category
df['loan_status'] = df['loan_status'].astype('bool')  


In [ ]:
df.info()

In [ ]:
df.cb_person_default_on_file.unique()

In [ ]:
## Save and export the cleaned data in a csv file:
df.to_csv('/Users/souadmouajel/Desktop/Ironhack/lab-sessions/week-8/Ironhack-Final-Project/Data/clean_data/cleaned_loan_data.csv', index=False)


In [ ]:
df['loan_status'].unique()

In [ ]:
df.loan_int_rate.describe()